# Stellium Analysis Cookbook

This notebook demonstrates the data analysis capabilities of the `stellium.analysis` module.

The analysis module provides tools for:
- **Batch chart calculation** - Calculate 100s-1000s of charts efficiently
- **DataFrame conversion** - Export charts to pandas DataFrames
- **Research queries** - Filter charts by astrological criteria
- **Statistical analysis** - Aggregate statistics across chart collections
- **Export utilities** - Save to CSV, JSON, Parquet

## Installation

The analysis module requires pandas (optional dependency):

```bash
pip install stellium[analysis]
```

In [1]:
# Imports
import pandas as pd

from stellium.analysis import (
    BatchCalculator,
    ChartQuery,
    ChartStats,
    aspects_to_dataframe,
    charts_to_dataframe,
    export_csv,
    export_json,
    positions_to_dataframe,
)
from stellium.core.native import Native
from stellium.engines.patterns import AspectPatternAnalyzer

---

## 1. BatchCalculator

Efficiently calculate many charts at once. `BatchCalculator` supports:
- Loading from the NotableRegistry (with filters)
- Loading from a list of Native objects
- Generator-based calculation (memory efficient)
- Progress callbacks

### 1.1 From NotableRegistry

Load charts from the built-in database of notable births and events.

In [2]:
# Calculate charts for all notables in the registry
all_charts = BatchCalculator.from_registry().calculate_all()
print(f"Calculated {len(all_charts)} charts")

Calculated 146 charts


In [3]:
# Filter by category
scientist_charts = BatchCalculator.from_registry(category="scientist").calculate_all()
print(f"Calculated {len(scientist_charts)} scientist charts")

Calculated 14 scientist charts


In [4]:
# Multiple filters: verified high-quality data only
verified_charts = BatchCalculator.from_registry(
    verified=True, data_quality="AA"
).calculate_all()
print(f"Calculated {len(verified_charts)} verified AA-quality charts")

Calculated 23 verified AA-quality charts


### 1.2 From Native Objects

Calculate charts from your own data.

In [5]:
# Create sample Native objects
sample_natives = [
    Native("2000-01-01 12:00", "New York, NY", name="Person A"),
    Native("1995-06-21 08:30", "Los Angeles, CA", name="Person B"),
    Native("1988-12-15 22:00", "Chicago, IL", name="Person C"),
    Native("1975-03-20 06:00", "Seattle, WA", name="Person D"),
    Native("1960-09-10 14:30", "Miami, FL", name="Person E"),
]

# Calculate charts
custom_charts = BatchCalculator.from_natives(sample_natives).calculate_all()
print(f"Calculated {len(custom_charts)} custom charts")

Calculated 5 custom charts


### 1.3 With Aspects and Analyzers

Configure the calculation with aspect detection and pattern analysis.

In [6]:
# Calculate with aspects and pattern detection
charts_with_aspects = (
    BatchCalculator.from_natives(sample_natives)
    .with_aspects()  # Enable aspect calculation
    .add_analyzer(AspectPatternAnalyzer())  # Detect Grand Trines, T-Squares, etc.
    .calculate_all()
)

# Check aspects for first chart
print(f"First chart has {len(charts_with_aspects[0].aspects)} aspects")

First chart has 46 aspects


### 1.4 Progress Tracking

Track progress for long-running batch calculations.

In [7]:
# Define a progress callback
def show_progress(current, total, name):
    print(f"  [{current}/{total}] Calculating: {name}")


# Calculate with progress tracking
print("Calculating charts with progress:")
charts = (
    BatchCalculator.from_natives(sample_natives[:3])  # Just first 3 for demo
    .with_progress(show_progress)
    .calculate_all()
)

Calculating charts with progress:
  [1/3] Calculating: Person A
  [2/3] Calculating: Person B
  [3/3] Calculating: Person C


### 1.5 Generator Mode (Memory Efficient)

For very large datasets, use the generator to process one chart at a time.

In [8]:
# Process charts one at a time (memory efficient)
batch = BatchCalculator.from_natives(sample_natives)

sun_signs = []
for chart in batch.calculate():
    sun = chart.get_object("Sun")
    sun_signs.append(sun.sign)

print(f"Sun signs: {sun_signs}")

Sun signs: ['Capricorn', 'Gemini', 'Sagittarius', 'Pisces', 'Virgo']


---

## 2. DataFrame Conversion

Convert charts to pandas DataFrames for analysis. Three schemas are available:
- **charts_to_dataframe**: One row per chart (chart-level data)
- **positions_to_dataframe**: One row per position (planet positions)
- **aspects_to_dataframe**: One row per aspect

### 2.1 Chart-Level DataFrame

One row per chart with summary data.

In [9]:
# Convert charts to DataFrame
df = charts_to_dataframe(custom_charts)
print(f"DataFrame shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")

DataFrame shape: (5, 33)

Columns: ['chart_id', 'name', 'datetime_utc', 'julian_day', 'latitude', 'longitude', 'location_name', 'sun_longitude', 'sun_sign', 'sun_sign_degree', 'moon_longitude', 'moon_sign', 'moon_sign_degree', 'moon_phase', 'moon_illumination', 'asc_longitude', 'asc_sign', 'mc_longitude', 'mc_sign', 'fire_count', 'earth_count', 'air_count', 'water_count', 'cardinal_count', 'fixed_count', 'mutable_count', 'sect', 'retrograde_count', 'has_grand_trine', 'has_t_square', 'has_grand_cross', 'has_yod', 'has_stellium']


In [10]:
# View the data
df[
    [
        "name",
        "sun_sign",
        "moon_sign",
        "asc_sign",
        "fire_count",
        "earth_count",
        "air_count",
        "water_count",
    ]
]

,name,sun_sign,moon_sign,asc_sign,fire_count,earth_count,air_count,water_count
0,Person A,Capricorn,Scorpio,Aries,3,3,3,1
1,Person B,Gemini,Aries,Leo,2,3,3,2
2,Person C,Sagittarius,Pisces,Virgo,2,5,0,3
3,Person D,Pisces,Gemini,Aquarius,2,1,3,4
4,Person E,Virgo,Taurus,Sagittarius,2,5,2,1


In [11]:
# Sun sign distribution
df["sun_sign"].value_counts()

sun_sign
Capricorn      1
Gemini         1
Sagittarius    1
Pisces         1
Virgo          1
Name: count, dtype: int64

### 2.2 Position-Level DataFrame

One row per celestial position - useful for analyzing specific planets across many charts.

In [12]:
# Convert to positions DataFrame
positions_df = positions_to_dataframe(custom_charts)
print(f"DataFrame shape: {positions_df.shape}")
positions_df.head(10)

DataFrame shape: (100, 13)


,chart_id,chart_name,object_name,object_type,longitude,latitude,sign,sign_degree,house,speed,is_retrograde,declination,is_out_of_bounds
0,f132b9b45f72,Person A,Sun,planet,280.580550,0.000226,Capricorn,10.580550,10,1.019454,False,-23.015803,False
1,f132b9b45f72,Person A,Saturn,planet,40.391562,-2.443869,Taurus,10.391562,1,-0.019565,True,12.614429,False
2,f132b9b45f72,Person A,Venus,planet,241.816804,2.060493,Sagittarius,1.816804,8,1.209279,False,-18.504544,False
3,f132b9b45f72,Person A,True Node,node,123.942397,0.000000,Leo,3.942397,5,-0.057228,True,19.267210,False
4,f132b9b45f72,Person A,Uranus,planet,314.819647,-0.658282,Aquarius,14.819647,11,0.050436,False,-17.017210,False
5,f132b9b45f72,Person A,Mars,planet,328.124329,-1.065195,Aquarius,28.124329,11,0.775680,False,-13.124256,False
6,f132b9b45f72,Person A,Jupiter,planet,25.261622,-1.261117,Aries,25.261622,1,0.041462,False,8.598366,False
7,f132b9b45f72,Person A,Pluto,planet,251.462070,10.855539,Sagittarius,11.462070,8,0.035101,False,-11.394913,False
8,f132b9b45f72,Person A,Chiron,asteroid,251.641339,4.073011,Sagittarius,11.641339,8,0.114174,False,-18.142799,False
9,f132b9b45f72,Person A,Mean Apogee,point,263.487444,3.417425,Sagittarius,23.487444,9,0.111329,False,-19.864088,False


In [13]:
# Filter to planets only
planets_df = positions_df[positions_df["object_type"] == "planet"]
print(f"Planet positions: {len(planets_df)}")

# Sign distribution for all planets
planets_df["sign"].value_counts()

Planet positions: 50


sign
Capricorn      9
Sagittarius    6
Scorpio        6
Gemini         5
Taurus         4
Aquarius       4
Aries          4
Pisces         4
Virgo          4
Libra          2
Cancer         1
Leo            1
Name: count, dtype: int64

In [14]:
# Retrograde analysis
retrograde_df = planets_df[planets_df["is_retrograde"]]
print(f"Retrograde positions: {len(retrograde_df)}")
retrograde_df[["chart_name", "object_name", "sign", "is_retrograde"]]

Retrograde positions: 10


,chart_name,object_name,sign,is_retrograde
1,Person A,Saturn,Taurus,True
24,Person B,Uranus,Capricorn,True
26,Person B,Jupiter,Sagittarius,True
27,Person B,Pluto,Scorpio,True
32,Person B,Neptune,Capricorn,True
46,Person C,Jupiter,Taurus,True
64,Person D,Uranus,Scorpio,True
67,Person D,Pluto,Libra,True
72,Person D,Neptune,Sagittarius,True
81,Person E,Saturn,Capricorn,True


### 2.3 Aspect-Level DataFrame

One row per aspect - useful for aspect frequency analysis.

In [15]:
# Convert to aspects DataFrame
aspects_df = aspects_to_dataframe(charts_with_aspects)
print(f"DataFrame shape: {aspects_df.shape}")
aspects_df.head(10)

DataFrame shape: (248, 9)


,chart_id,chart_name,object1,object2,aspect_name,aspect_degree,orb,is_applying,aspect_type
0,f132b9b45f72,Person A,Sun,Saturn,Trine,120.0,0.188987,False,longitude
1,f132b9b45f72,Person A,Sun,Moon,Sextile,60.0,5.235663,False,longitude
2,f132b9b45f72,Person A,Sun,MC,Conjunction,0.0,0.108874,None,longitude
3,f132b9b45f72,Person A,Sun,Vertex,Square,90.0,2.328398,None,longitude
4,f132b9b45f72,Person A,Sun,RAMC,Conjunction,0.0,0.809169,None,longitude
5,f132b9b45f72,Person A,Saturn,True Node,Square,90.0,6.449166,False,longitude
6,f132b9b45f72,Person A,Saturn,Uranus,Square,90.0,4.428085,False,longitude
7,f132b9b45f72,Person A,Saturn,Moon,Opposition,180.0,5.424650,False,longitude
8,f132b9b45f72,Person A,Saturn,Neptune,Square,90.0,7.191162,True,longitude
9,f132b9b45f72,Person A,Saturn,South Node,Square,90.0,6.449166,True,longitude


In [16]:
# Most common aspects
aspects_df["aspect_name"].value_counts()

aspect_name
Square         70
Sextile        59
Trine          58
Conjunction    34
Opposition     27
Name: count, dtype: int64

In [17]:
# Sun-Moon aspects
sun_moon = aspects_df[
    ((aspects_df["object1"] == "Sun") & (aspects_df["object2"] == "Moon"))
    | ((aspects_df["object1"] == "Moon") & (aspects_df["object2"] == "Sun"))
]
print(f"Sun-Moon aspects: {len(sun_moon)}")
sun_moon[["chart_name", "aspect_name", "orb"]]

Sun-Moon aspects: 4


,chart_name,aspect_name,orb
1,Person A,Sextile,5.235663
97,Person C,Square,0.918646
154,Person D,Square,3.667081
207,Person E,Trine,5.768603


---

## 3. ChartQuery

Filter chart collections by astrological criteria using a fluent API.

Query methods include:
- `where_sun()`, `where_moon()`, `where_planet()`, `where_angle()`
- `where_aspect()`, `where_pattern()`
- `where_element_dominant()`, `where_modality_dominant()`
- `where_sect()`, `where_custom()`

### 3.1 Basic Filtering

In [18]:
# Find charts with Sun in fire signs
fire_sun_charts = (
    ChartQuery(custom_charts).where_sun(sign=["Aries", "Leo", "Sagittarius"]).results()
)
print(f"Charts with Sun in fire signs: {len(fire_sun_charts)}")
for chart in fire_sun_charts:
    print(f"  - {chart.metadata.get('name')}: Sun in {chart.get_object('Sun').sign}")

Charts with Sun in fire signs: 1
  - Person C: Sun in Sagittarius


In [19]:
# Find charts with Mercury retrograde
mercury_rx = (
    ChartQuery(custom_charts).where_planet("Mercury", retrograde=True).results()
)
print(f"Charts with Mercury retrograde: {len(mercury_rx)}")

Charts with Mercury retrograde: 0


### 3.2 Chained Filters

Combine multiple criteria for complex queries.

In [20]:
# Find day charts with Sun in earth signs
results = (
    ChartQuery(custom_charts)
    .where_sun(sign=["Taurus", "Virgo", "Capricorn"])
    .where_sect("day")
    .results()
)
print(f"Day charts with Sun in earth signs: {len(results)}")

Day charts with Sun in earth signs: 0


In [21]:
# Find charts with Sun-Moon conjunction
conjunctions = (
    ChartQuery(charts_with_aspects)
    .where_aspect("Sun", "Moon", aspect="Conjunction", orb_max=10)
    .results()
)
print(f"Charts with Sun-Moon conjunction: {len(conjunctions)}")

Charts with Sun-Moon conjunction: 0


### 3.3 Element and Modality Dominance

In [22]:
# Find charts with dominant fire element (4+ planets)
fire_dominant = (
    ChartQuery(custom_charts).where_element_dominant("fire", min_count=4).results()
)
print(f"Charts with fire dominance: {len(fire_dominant)}")

# Find charts with dominant cardinal modality
cardinal_dominant = (
    ChartQuery(custom_charts).where_modality_dominant("cardinal", min_count=4).results()
)
print(f"Charts with cardinal dominance: {len(cardinal_dominant)}")

Charts with fire dominance: 0
Charts with cardinal dominance: 1


### 3.4 Custom Predicates

Use any custom function to filter charts.

In [23]:
# Find charts with more than 5 aspects
many_aspects = (
    ChartQuery(charts_with_aspects).where_custom(lambda c: len(c.aspects) > 5).results()
)
print(f"Charts with > 5 aspects: {len(many_aspects)}")

Charts with > 5 aspects: 5


In [24]:
# Find charts where Moon is in the same sign as Sun
sun_moon_same = (
    ChartQuery(custom_charts)
    .where_custom(lambda c: c.get_object("Sun").sign == c.get_object("Moon").sign)
    .results()
)
print(f"Charts with Sun and Moon in same sign: {len(sun_moon_same)}")
for chart in sun_moon_same:
    print(f"  - {chart.metadata.get('name')}: both in {chart.get_object('Sun').sign}")

Charts with Sun and Moon in same sign: 0


### 3.5 Result Methods

In [25]:
query = ChartQuery(custom_charts).where_sun(
    sign=["Aries", "Taurus", "Gemini", "Cancer"]
)

# Get count
print(f"Count: {query.count()}")

# Get first result
first = query.first()
if first:
    print(f"First match: {first.metadata.get('name')}")

# Convert to DataFrame
df = query.to_dataframe()
df[["name", "sun_sign", "moon_sign"]]

Count: 1
First match: Person B


,name,sun_sign,moon_sign
0,Person B,Gemini,Aries


---

## 4. ChartStats

Compute aggregate statistics across chart collections.

In [26]:
# Create stats object
stats = ChartStats(custom_charts)
print(f"Analyzing {stats.chart_count} charts")

Analyzing 5 charts


### 4.1 Element and Modality Distribution

In [27]:
# Element distribution (normalized proportions)
elements = stats.element_distribution()
print("Element Distribution:")
for element, proportion in elements.items():
    print(f"  {element.title()}: {proportion:.1%}")

Element Distribution:
  Fire: 22.0%
  Earth: 34.0%
  Air: 22.0%
  Water: 22.0%


In [28]:
# Modality distribution
modalities = stats.modality_distribution()
print("Modality Distribution:")
for modality, proportion in modalities.items():
    print(f"  {modality.title()}: {proportion:.1%}")

Modality Distribution:
  Cardinal: 32.0%
  Fixed: 30.0%
  Mutable: 38.0%


### 4.2 Sign Distribution

In [29]:
# Sun sign distribution
sun_signs = stats.sign_distribution("Sun")
print("Sun Sign Distribution:")
for sign, count in sun_signs.items():
    if count > 0:
        print(f"  {sign}: {count}")

Sun Sign Distribution:
  Gemini: 1
  Virgo: 1
  Sagittarius: 1
  Capricorn: 1
  Pisces: 1


In [30]:
# Moon sign distribution
moon_signs = stats.sign_distribution("Moon")
print("Moon Sign Distribution:")
for sign, count in moon_signs.items():
    if count > 0:
        print(f"  {sign}: {count}")

Moon Sign Distribution:
  Aries: 1
  Taurus: 1
  Gemini: 1
  Scorpio: 1
  Pisces: 1


### 4.3 House Distribution

In [31]:
# Sun house distribution
sun_houses = stats.house_distribution("Sun")
print("Sun House Distribution:")
for house, count in sun_houses.items():
    if count > 0:
        print(f"  House {house}: {count}")

Sun House Distribution:
  House 1: 1
  House 4: 1
  House 9: 1
  House 10: 1
  House 11: 1


### 4.4 Aspect Frequency

In [32]:
# Create stats from charts with aspects
stats_aspects = ChartStats(charts_with_aspects)

# Overall aspect frequency
aspect_freq = stats_aspects.aspect_frequency()
print("Aspect Frequency:")
for aspect, count in list(aspect_freq.items())[:5]:
    print(f"  {aspect}: {count}")

Aspect Frequency:
  Square: 70
  Sextile: 59
  Trine: 58
  Conjunction: 34
  Opposition: 27


In [33]:
# Aspect frequency between specific planets
sun_moon_aspects = stats_aspects.aspect_pair_frequency("Sun", "Moon")
print("Sun-Moon Aspect Frequency:")
for aspect, count in sun_moon_aspects.items():
    print(f"  {aspect}: {count}")

Sun-Moon Aspect Frequency:
  Sextile: 1
  Square: 2
  Trine: 1


### 4.5 Retrograde Frequency

In [34]:
# Retrograde frequency by planet
retro_freq = stats.retrograde_frequency()
print("Retrograde Counts by Planet:")
for planet, count in retro_freq.items():
    print(f"  {planet}: {count}")

Retrograde Counts by Planet:
  Saturn: 2
  Uranus: 2
  Jupiter: 2
  Pluto: 2
  Neptune: 2


In [35]:
# Retrograde rate (normalized)
retro_rate = stats.retrograde_frequency(normalize=True)
print("Retrograde Rate by Planet:")
for planet, rate in retro_rate.items():
    print(f"  {planet}: {rate:.1%}")

Retrograde Rate by Planet:
  Sun: 0.0%
  Saturn: 40.0%
  Venus: 0.0%
  Uranus: 40.0%
  Mars: 0.0%
  Jupiter: 40.0%
  Pluto: 40.0%
  Mercury: 0.0%
  Moon: 0.0%
  Neptune: 40.0%


### 4.6 Sect Distribution

In [36]:
# Day vs night charts
sect_dist = stats.sect_distribution()
print("Sect Distribution:")
for sect, count in sect_dist.items():
    print(f"  {sect.title()}: {count}")

Sect Distribution:
  Night: 3
  Day: 2


### 4.7 Cross-Tabulation

Create contingency tables to analyze relationships between variables.

In [37]:
# Sun sign vs Moon sign cross-tabulation
crosstab = stats.cross_tab("sun_sign", "moon_sign")
print("Sun Sign vs Moon Sign:")
crosstab

Sun Sign vs Moon Sign:


moon_sign,Aries,Gemini,Pisces,Scorpio,Taurus
sun_sign,,,,,
Capricorn,0,0,0,1,0
Gemini,1,0,0,0,0
Pisces,0,1,0,0,0
Sagittarius,0,0,1,0,0
Virgo,0,0,0,0,1


In [38]:
# Sun sign vs sect
sun_sect = stats.cross_tab("sun_sign", "sect")
print("Sun Sign vs Sect:")
sun_sect

Sun Sign vs Sect:


sect,day,night
sun_sign,,
Capricorn,0,1
Gemini,0,1
Pisces,1,0
Sagittarius,1,0
Virgo,0,1


### 4.8 Summary Report

In [39]:
# Get comprehensive summary
summary = stats.summary()

print(f"Chart Count: {summary['chart_count']}")
print(f"\nElement Distribution: {summary['element_distribution']}")
print(f"\nModality Distribution: {summary['modality_distribution']}")
print(f"\nSect Distribution: {summary['sect_distribution']}")

Chart Count: 5

Element Distribution: {'fire': 0.22, 'earth': 0.34, 'air': 0.22, 'water': 0.22}

Modality Distribution: {'cardinal': 0.32, 'fixed': 0.3, 'mutable': 0.38}

Sect Distribution: {'night': 3, 'day': 2}


---

## 5. Export Utilities

Save chart data to files for external analysis.

(Examples in this notebook use `tempfile` for cookbook purposes; replace with the actual directory you want.)

### 5.1 CSV Export

In [40]:
import tempfile
from pathlib import Path

# Export chart-level data
with tempfile.TemporaryDirectory() as tmpdir:
    # Charts CSV
    charts_path = Path(tmpdir) / "charts.csv"
    export_csv(custom_charts, charts_path, schema="charts")
    print(f"Exported charts to {charts_path}")

    # Read back and display
    df = pd.read_csv(charts_path)
    print(f"Shape: {df.shape}")
    df.head()

Exported charts to /var/folders/bn/szhh8wcd5hj4jt5qw5ntqtp80000gn/T/tmpidiiptee/charts.csv
Shape: (5, 33)


In [42]:
# Export positions
with tempfile.TemporaryDirectory() as tmpdir:
    positions_path = Path(tmpdir) / "positions.csv"
    export_csv(custom_charts, positions_path, schema="positions")

    df = pd.read_csv(positions_path)
    print(f"Positions CSV shape: {df.shape}")
    df.head()

Positions CSV shape: (100, 13)


### 5.2 JSON Export

In [43]:
import json

with tempfile.TemporaryDirectory() as tmpdir:
    # Standard JSON array
    json_path = Path(tmpdir) / "charts.json"
    export_json(custom_charts, json_path)

    with open(json_path) as f:
        data = json.load(f)

    print(f"Exported {len(data)} charts to JSON")
    print(f"Keys in first chart: {list(data[0].keys())[:10]}...")

Exported 5 charts to JSON
Keys in first chart: ['chart_tags', 'datetime', 'location', 'house_systems', 'default_house_system', 'house_placements', 'positions', 'aspects', 'declination_aspects', 'metadata']...


In [44]:
# JSON Lines format (for streaming large datasets)
with tempfile.TemporaryDirectory() as tmpdir:
    jsonl_path = Path(tmpdir) / "charts.jsonl"
    export_json(custom_charts, jsonl_path, lines=True)

    with open(jsonl_path) as f:
        lines = f.readlines()

    print(f"Exported {len(lines)} lines to JSONL")
    print(f"First line preview: {lines[0][:100]}...")

Exported 5 lines to JSONL
First line preview: {"chart_tags": [], "datetime": {"utc": "2000-01-01T17:00:00+00:00", "julian_date": 2451545.207594570...


---

## 6. Full Workflow Examples

Complete research workflows combining multiple features.

### 6.1 Research Question: Element Distribution in Scientists vs Artists

Compare element distributions between different categories.

In [45]:
# Calculate charts for scientists and artists
scientists = BatchCalculator.from_registry(category="scientist").calculate_all()
artists = BatchCalculator.from_registry(category="artist").calculate_all()

print(f"Scientists: {len(scientists)}")
print(f"Artists: {len(artists)}")

# Compare element distributions
if scientists and artists:
    sci_stats = ChartStats(scientists)
    art_stats = ChartStats(artists)

    print("\nElement Distribution Comparison:")
    print(f"{'Element':<12} {'Scientists':<15} {'Artists':<15}")
    print("-" * 42)

    sci_elements = sci_stats.element_distribution()
    art_elements = art_stats.element_distribution()

    for element in ["fire", "earth", "air", "water"]:
        sci_pct = sci_elements.get(element, 0)
        art_pct = art_elements.get(element, 0)
        print(f"{element.title():<12} {sci_pct:>12.1%}   {art_pct:>12.1%}")

Scientists: 14
Artists: 14

Element Distribution Comparison:
Element      Scientists      Artists        
------------------------------------------
Fire                23.6%          27.9%
Earth               21.4%          27.9%
Air                 22.1%          17.9%
Water               32.9%          26.4%


### 6.2 Research Question: Mercury Retrograde Frequency

Analyze how often Mercury is retrograde in a collection.

In [46]:
# Get all charts
all_charts = BatchCalculator.from_registry().calculate_all()

if all_charts:
    # Query for Mercury retrograde
    mercury_rx_charts = (
        ChartQuery(all_charts).where_planet("Mercury", retrograde=True).results()
    )

    total = len(all_charts)
    rx_count = len(mercury_rx_charts)

    print(f"Total charts: {total}")
    print(f"Mercury Rx charts: {rx_count}")
    print(f"Mercury Rx rate: {rx_count / total:.1%}")
    print("\n(Expected astronomical rate is ~19%)")

Total charts: 146
Mercury Rx charts: 26
Mercury Rx rate: 17.8%

(Expected astronomical rate is ~19%)


### 6.3 Research Question: Sun-Moon Aspect Distribution

Analyze the distribution of aspects between Sun and Moon.

In [47]:
# Calculate charts with aspects
charts = BatchCalculator.from_registry().with_aspects().calculate_all()

if charts:
    stats = ChartStats(charts)
    sun_moon_aspects = stats.aspect_pair_frequency("Sun", "Moon")

    print("Sun-Moon Aspect Distribution:")
    total_aspects = sum(sun_moon_aspects.values())
    for aspect, count in sorted(sun_moon_aspects.items(), key=lambda x: -x[1]):
        pct = count / total_aspects if total_aspects > 0 else 0
        print(f"  {aspect}: {count} ({pct:.1%})")

Sun-Moon Aspect Distribution:
  Trine: 17 (34.7%)
  Sextile: 14 (28.6%)
  Conjunction: 7 (14.3%)
  Square: 7 (14.3%)
  Opposition: 4 (8.2%)


### 6.4 Pipeline: Filter, Analyze, Export

A complete data pipeline example.

In [48]:
# 1. Calculate all charts with aspects
charts = (
    BatchCalculator.from_registry()
    .with_aspects()
    .add_analyzer(AspectPatternAnalyzer())
    .calculate_all()
)

print(f"Step 1: Calculated {len(charts)} charts")

# 2. Filter to fire-dominant charts
fire_charts = ChartQuery(charts).where_element_dominant("fire", min_count=4).results()

print(f"Step 2: Found {len(fire_charts)} fire-dominant charts")

# 3. Analyze the subset
if fire_charts:
    fire_stats = ChartStats(fire_charts)
    print("\nStep 3: Analysis of fire-dominant charts:")
    print(f"  Element distribution: {fire_stats.element_distribution()}")
    print(f"  Sect distribution: {fire_stats.sect_distribution()}")

# 4. Convert to DataFrame for further analysis
if fire_charts:
    df = charts_to_dataframe(fire_charts)
    print(f"\nStep 4: DataFrame created with {len(df)} rows")
    display(df[["name", "sun_sign", "fire_count", "sect"]].head())

Step 1: Calculated 146 charts
Step 2: Found 31 fire-dominant charts

Step 3: Analysis of fire-dominant charts:
  Element distribution: {'fire': 0.44516129032258067, 'earth': 0.15483870967741936, 'air': 0.1774193548387097, 'water': 0.22258064516129034}
  Sect distribution: {'day': 10, 'night': 21}

Step 4: DataFrame created with 31 rows


,name,sun_sign,fire_count,sect
0,Titanic Sinking,Aries,4,day
1,John F. Kennedy Assassination,Scorpio,4,night
2,Pearl Harbor Attack,Sagittarius,4,night
3,Indian Independence,Leo,5,day
4,Bill Gates,Scorpio,4,day


---

## Summary

The `stellium.analysis` module provides a complete toolkit for large-scale astrological data analysis:

| Component | Purpose |
|-----------|--------|
| `BatchCalculator` | Efficiently calculate many charts at once |
| `charts_to_dataframe()` | Convert to chart-level DataFrame |
| `positions_to_dataframe()` | Convert to position-level DataFrame |
| `aspects_to_dataframe()` | Convert to aspect-level DataFrame |
| `ChartQuery` | Filter charts by astrological criteria |
| `ChartStats` | Compute aggregate statistics |
| `export_csv()` | Export to CSV files |
| `export_json()` | Export to JSON or JSONL |

For more information, see the [Stellium documentation](https://stellium.readthedocs.io/).